# Technical note: Equality checking, hashing, pickling

## Equality vs is

`==` checks if two objects "should be considered equal", e.g. `myokit.Number(1) == myokit.Number(1)` should return `True`.

`is` checks if two variables point to the same object, e.g. `x is x` should return `True`, as should `y = x; x is y`. 

### id(), and why to just use is

- The line `x is y` is equivalent to `id(x) == id(y)`. 
- An object's id is unique **during its lifetime**. In the standard implementation, the value returned by id(x) is `&x`, the memory address of x.

This means that:
- If you store an objects id **but not the object**, and then want to check if you're seeing the same object again, you can't use `id(x) == stored_id`.
- **However**, in most cases you would simply store the object. As long as you have a reference to the object, its id will stay in use, and so `x is stored_x` will always return the correct answer.

### Literals

A line like `Number(1) is Number(1)` creates two new objects, each with their own id, and so returns False.
It gets a bit more complicated for literals, as Python tends to cache them: a line `1 is 1` retrieves two references to the same object (I think -- there might be further tricky details to make things fast) and so can return True.

This seems to hold for low ints and floats, but not for strings and very long integers:
```
>>> id(1)
140111174779120
>>> id(1)
140111174779120
>>> id(1)
140111174779120

>>> id(-1.234e-5)
140111173736976
>>> id(-1.234e-5)
140111173736976
>>> id(-1.234e-5)
140111173736976
```

```
>>> id('Hello')
140111173489136
>>> id('Hello')
140111173489328
>>> id('Hello')
140111173489136
```
```
>>> id(12345678901234567890)
140111173472384
>>> id(12345678901234567890)
140111173473296
>>> id(12345678901234567890)
140111173472288
```

## When should Myokit objects be "considered equal"

The following objects define an `__eq__` method in Myokit:

- Units & quantities
- Quantity
- Expression (Expression, Name, ALL OTHER LHS EXPR?)
- Equation
- Model
- Protocol

### Units & quantities





Note 1: Quantities might be merged with myokit.Number in [#798](https://github.com/MichaelClerx/myokit/issues/798).

Note 2: Units already have a preferred global representation, but might get a per-object one in [#783](https://github.com/MichaelClerx/myokit/issues/783).

### Models

**At the moment** (2022-02-03), models are considered equal if they are the same object, or if

- they have the same set of reserved unames (which are strings, so immutable and easy to compare), and
- they have the same set of reserved uname prefixes (strings again), and
- the output of their `code()` methods is the same. 

There are some pros and cons:

- **pro** If you load the same model twice, the models are equal
- **pro** Once you modify a model, it's no longer equal
- **con** The unames are not something many/most users will remember about (pro: but that perhaps means they won't use them?)
- **con** Because components and variables don't have a custom `__eq__`, two models that are "considered equal" will consists of components and variables that are *not* considered equal. 

**The final point may hold for expressions too. See below**

## Hashing and equality

Sets and dicts in Python are based on hash maps. To make objects useable as keys in a dict or items in a set, they need to implement a *hash function* that returns an **almost unique integer**. Look-ups in a set or dict start with a quick hash-based jump, followed by a "proper" check using `==`. As a result, the `__hash__` and `__eq__` methods of user-defined classes have [some restrictions](https://docs.python.org/3/reference/datamodel.html#object.__hash__):

**Default implementations use `is`**.
By default, `x.__eq__(y)` returns `x == y`, and `hash(x)` returns "an appropriate value such that x == y implies both that x is y and hash(x) == hash(y)". So if you leave hash and eq alone, your objects will be hashable, but with an "is" condition. In other words, `myokit.Number(1) in {myokit.Number(1)}` will return False because `id(myokit.Number(1)) != id(myokit.Number(1))`.

**Overriding eq removes default hash**.
If you override `__eq__` but not `__hash__`, Python will automatically set `YourClass.__hash__ = None`, rendering your object unhashable.

**Overriding hash? Then do eq too**.
If you override `__hash__`, you **must** also provide an appropriate `__eq__` function.

**Hashes must be immutable**.
The value returned by an object's `__hash__` must stay the same during its lifetime. So in general you should only implement `__hash__` for immutable objects.

### Hashing in Myokit

The following Myokit classes override `__hash__`:

- Unit
- Quantity
- Equation
- Expression (the base class)

## Tail recursion optimisation

Is something that Python doesn't have, that _may_ apply to expressions (although we're usually OK with them being a bit slow-ish, as they are immutable), and can be hacked in with decorators: https://chrispenner.ca/posts/python-tail-recursion



## Pickling

The following objects in Myokit have extra functions related to pickling & unpickling:

- Model
- Protocol
- Simulation
- LegacySimulation
- **TODO** Expressions?